In [1]:
import pandas as pd
df1=pd.read_csv('Prediction.csv')
df1.head()

,Unnamed: 0,Age,Salary,Designation,Department,Key_skills,Exp_in_years
0,0,25,71494,Product Operations Analyst.Associate.,IT Product Management & Ops,team skills communication skills analytical sk...,0.58
1,1,26,39971,Platform Operations Engineer.Associate.,Platform Operations,ppc logistics inventory management supply chai...,0.50
2,2,24,77705,Platform Operations Engineer.Associate.,Platform Operations,hr analytics employee engagement training succ...,0.67
3,3,26,40164,Platform Operations Engineer.Associate.,Platform Operations,sql javascript automation python ruby analytic...,0.50
4,4,29,143963,Software Engineer.Senior.,Enterprise Access Engineering,accounting finance cash flow financial plannin...,2.08


In [3]:
df1.rename(columns={'Unnamed: 0':'del'},inplace=True)

In [4]:
df1.drop(columns=['del'],inplace=True)

In [5]:
df1.shape

(14416, 6)

In [6]:
df1.describe()

,Age,Salary,Exp_in_years
count,14416.000000,14416.000000,14416.000000
mean,31.516648,122454.117092,4.031448
std,5.223815,58578.003087,2.969199
min,21.000000,256.000000,0.500000
25%,28.000000,84064.000000,1.920000
50%,31.000000,109753.000000,3.250000
75%,35.000000,145906.000000,5.500000
max,56.000000,660810.000000,15.830000


In [7]:
df1.isnull().sum()

Age             0
Salary          0
Designation     0
Department      0
Key_skills      8
Exp_in_years    0
dtype: int64

In [8]:
df1.Key_skills=df1.Key_skills.fillna('')

## Using Label Encoder to change categorical features into numerical form

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
l=LabelEncoder()
label=l.fit_transform(df1["Designation"])
label1=l.fit_transform(df1["Department"])
label2=l.fit_transform(df1["Key_skills"])

In [11]:
df1['Designation']=label
df1['Department']=label1
df1['Key_skills']=label2

In [12]:
df1.head()

,Age,Salary,Designation,Department,Key_skills,Exp_in_years
0,25,71494,244,72,1619,0.58
1,26,39971,231,117,1087,0.50
2,24,77705,231,117,694,0.67
3,26,40164,231,117,1542,0.50
4,29,143963,316,41,25,2.08


In [13]:
x=df1.drop('Salary',axis='columns')
y=df1.Salary
x

,Age,Designation,Department,Key_skills,Exp_in_years
0,25,244,72,1619,0.58
1,26,231,117,1087,0.50
2,24,231,117,694,0.67
3,26,231,117,1542,0.50
4,29,316,41,25,2.08
...,...,...,...,...,...
14411,29,371,19,358,2.83
14412,27,309,57,1398,2.75
14413,29,23,83,733,1.25
14414,47,124,26,347,2.50


In [14]:
df1.dtypes

Age               int64
Salary            int64
Designation       int32
Department        int32
Key_skills        int32
Exp_in_years    float64
dtype: object

In [17]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'Decision_tree': {
        'model': tree.DecisionTreeClassifier(),
        'params' : {
            'criterion' :['gini', 'entropy'],
            'splitter' :['best', 'random']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(),
        'params' : {
            'C': [1,5,10],
            'kernel': ['linear', 'poly', 'rbf']
        }
    }
}

In [18]:
from sklearn.model_selection import GridSearchCV

In [17]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x,y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,Decision_tree,1.000000,"{'criterion': 'gini', 'splitter': 'best'}"
1,random_forest,1.000000,{'n_estimators': 5}
2,logistic_regression,0.492856,{'C': 10}


In [52]:
from sklearn.model_selection import cross_val_score

In [26]:
cross_val_score(RandomForestClassifier(n_estimators=5),x, y, cv=5)

array([1., 1., 1., 1., 1.])

In [54]:
cross_val_score(tree.DecisionTreeClassifier(),x, y, cv=5)

array([1., 1., 1., 1., 1.])

In [56]:
cross_val_score(LogisticRegression(C=10),x,y, cv=5)

C:\Users\SAI\anaconda3\envs\minor_project\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SAI\anaconda3\envs\minor_project\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

array([0.31276006, 0.2750607 , 0.26916407, 0.27679501, 0.26638918])

### Logistic Reggresion analysis

In [71]:
from sklearn.linear_model import LogisticRegression
lr_res = GridSearchCV(LogisticRegression(), { 
            'C': [1,5,10]
}, cv=5, return_train_score=False)
lr_res.fit(x, y)
lr_res.cv_results_

C:\Users\SAI\anaconda3\envs\minor_project\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SAI\anaconda3\envs\minor_project\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

{'mean_fit_time': array([116.16470351, 117.68203797, 174.62199602]),
 'std_fit_time': array([ 3.83982161, 12.50978146, 52.17109221]),
 'mean_score_time': array([0.04870801, 0.05332794, 0.07785568]),
 'std_score_time': array([0.00421662, 0.00697053, 0.02330507]),
 'param_C': masked_array(data=[1, 5, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1}, {'C': 5}, {'C': 10}],
 'split0_test_score': array([0.29854369, 0.29403606, 0.31276006]),
 'split1_test_score': array([0.25841138, 0.27436698, 0.2750607 ]),
 'split2_test_score': array([0.28511967, 0.29136316, 0.26916407]),
 'split3_test_score': array([0.22545959, 0.24592438, 0.27679501]),
 'split4_test_score': array([0.19632327, 0.26569546, 0.26638918]),
 'mean_test_score': array([0.25277152, 0.27427721, 0.2800338 ]),
 'std_test_score': array([0.03769787, 0.01766284, 0.01679617]),
 'rank_test_score': array([3, 2, 1])}

In [72]:
result_lr = pd.DataFrame(lr_res.cv_results_)
result_lr

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,116.164704,3.839822,0.048708,0.004217,1,{'C': 1},0.298544,0.258411,0.285120,0.225460,0.196323,0.252772,0.037698,3
1,117.682038,12.509781,0.053328,0.006971,5,{'C': 5},0.294036,0.274367,0.291363,0.245924,0.265695,0.274277,0.017663,2
2,174.621996,52.171092,0.077856,0.023305,10,{'C': 10},0.312760,0.275061,0.269164,0.276795,0.266389,0.280034,0.016796,1


In [74]:
lr_result = result_lr.drop(columns=['mean_fit_time','std_fit_time','mean_score_time','std_score_time','param_C'])
lr_result

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,{'C': 1},0.298544,0.258411,0.285120,0.225460,0.196323,0.252772,0.037698,3
1,{'C': 5},0.294036,0.274367,0.291363,0.245924,0.265695,0.274277,0.017663,2
2,{'C': 10},0.312760,0.275061,0.269164,0.276795,0.266389,0.280034,0.016796,1


### Decision Tree analysis

In [75]:
from sklearn import tree
dt_res = GridSearchCV(tree.DecisionTreeClassifier(), { 
             'criterion' :['gini', 'entropy'],
            'splitter' :['best', 'random']
}, cv=5, return_train_score=False)
dt_res.fit(x, y)
dt_res.cv_results_

{'mean_fit_time': array([1.16978064, 0.19334698, 0.98584733, 0.13628707]),
 'std_fit_time': array([0.07312849, 0.00466218, 0.02580701, 0.02392024]),
 'mean_score_time': array([0.04060946, 0.03995266, 0.03523645, 0.02948542]),
 'std_score_time': array([0.00167351, 0.00337272, 0.00354835, 0.00833298]),
 'param_criterion': masked_array(data=['gini', 'gini', 'entropy', 'entropy'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_splitter': masked_array(data=['best', 'random', 'best', 'random'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'gini', 'splitter': 'best'},
  {'criterion': 'gini', 'splitter': 'random'},
  {'criterion': 'entropy', 'splitter': 'best'},
  {'criterion': 'entropy', 'splitter': 'random'}],
 'split0_test_score': array([1., 1., 1., 1.]),
 'split1_test_score': array([1., 1., 1., 1.]),
 'split2_test_score': array([1., 1., 1., 1.]),
 

In [76]:
result_dt = pd.DataFrame(dt_res.cv_results_)
result_dt

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.169781,0.073128,0.040609,0.001674,gini,best,"{'criterion': 'gini', 'splitter': 'best'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.193347,0.004662,0.039953,0.003373,gini,random,"{'criterion': 'gini', 'splitter': 'random'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.985847,0.025807,0.035236,0.003548,entropy,best,"{'criterion': 'entropy', 'splitter': 'best'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.136287,0.023920,0.029485,0.008333,entropy,random,"{'criterion': 'entropy', 'splitter': 'random'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [77]:
dt_result = result_dt.drop(columns=['mean_fit_time','std_fit_time','mean_score_time','std_score_time','param_criterion','param_splitter'])
dt_result

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,"{'criterion': 'gini', 'splitter': 'best'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,"{'criterion': 'gini', 'splitter': 'random'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,"{'criterion': 'entropy', 'splitter': 'best'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,"{'criterion': 'entropy', 'splitter': 'random'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


### Random Forest Analysis

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=5)
rf.fit(x,y)
print(rf.predict([[26,231,117,1087,0.50]]))

[39971]


In [20]:
from sklearn.model_selection import GridSearchCV
rf_res = GridSearchCV(RandomForestClassifier(), {
    'n_estimators':[1,5,10]
}, cv=5, return_train_score=False)
rf_res.fit(x, y)
rf_res.cv_results_

{'mean_fit_time': array([0.49680281, 2.68183293, 5.24284263]),
 'std_fit_time': array([0.08468575, 0.14830709, 0.85587694]),
 'mean_score_time': array([0.15346036, 0.53374796, 1.03368096]),
 'std_score_time': array([0.01502092, 0.04693138, 0.08326927]),
 'param_n_estimators': masked_array(data=[1, 5, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 1}, {'n_estimators': 5}, {'n_estimators': 10}],
 'split0_test_score': array([0.99791956, 1.        , 1.        ]),
 'split1_test_score': array([0.9982657, 1.       , 1.       ]),
 'split2_test_score': array([0.99687825, 1.        , 1.        ]),
 'split3_test_score': array([0.99791883, 1.        , 1.        ]),
 'split4_test_score': array([0.99895942, 1.        , 1.        ]),
 'mean_test_score': array([0.99798835, 1.        , 1.        ]),
 'std_test_score': array([0.00067257, 0.        , 0.        ]),
 'rank_test_score': array([3, 1, 1])}

In [21]:
result = pd.DataFrame(rf_res.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.496803,0.084686,0.153460,0.015021,1,{'n_estimators': 1},0.99792,0.998266,0.996878,0.997919,0.998959,0.997988,0.000673,3
1,2.681833,0.148307,0.533748,0.046931,5,{'n_estimators': 5},1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
2,5.242843,0.855877,1.033681,0.083269,10,{'n_estimators': 10},1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1


In [22]:
rf_result = result.drop(columns=['mean_fit_time','std_fit_time','mean_score_time','std_score_time','param_n_estimators'])
rf_result

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,{'n_estimators': 1},0.99792,0.998266,0.996878,0.997919,0.998959,0.997988,0.000673,3
1,{'n_estimators': 5},1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
2,{'n_estimators': 10},1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1


### Export code

In [23]:
import pickle

In [25]:
filename = 'finalized_model.sav'
pickle.dump(rf, open(filename, 'wb'))

In [26]:
loaded_model = pickle.load(open(filename, 'rb'))